## Autor: Patryk Klytta
ZADANIE 1

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
EPOCHS = 50
BATCH_SIZE = 16
OUTPUT_SEQUENCE_LEN = 7
codes = [' ', 'I', 'V', 'X', 'L', 'C', 'D', 'M']

In [3]:
def int_to_roman(num):
    val = [1000, 900, 500, 400, 100, 90, 50, 40, 10, 9, 5, 4, 1]
    syms = ['M', 'CM', 'D', 'CD', 'C', 'XC', 'L', 'XL', 'X', 'IX', 'V', 'IV', 'I']
    roman = ''
    i = 0
    while num > 0:
        for _ in range(num // val[i]):
            roman += syms[i]
            num -= val[i]
        i += 1
    return roman

In [4]:
num_samples = 1000
trainSamples = np.random.randint(1, 201, size=num_samples)
trainLabels = [int_to_roman(num) for num in trainSamples]
trainSamples_padded = pad_sequences(trainSamples.reshape(-1, 1), maxlen=OUTPUT_SEQUENCE_LEN, padding='post')


In [5]:
nlabels = np.zeros((len(trainLabels), OUTPUT_SEQUENCE_LEN, len(codes)))
for i, label in enumerate(trainLabels):
    for j in range(OUTPUT_SEQUENCE_LEN):
        if j < len(label):
            index = codes.index(label[j])
            nlabels[i][j][index] = 1
        else:
            nlabels[i][j][0] = 1

In [6]:
model = Sequential([
    Embedding(input_dim=201, output_dim=16, input_length=OUTPUT_SEQUENCE_LEN),
    LSTM(64, return_sequences=True),
    Dense(len(codes), activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

#model.summary()

In [9]:
model.fit(trainSamples_padded, nlabels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)


Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3469 - loss: 1.8312
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5038 - loss: 1.3086
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5413 - loss: 1.2089
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5949 - loss: 1.1380
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6812 - loss: 0.9186
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7176 - loss: 0.8100
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7418 - loss: 0.7484
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7516 - loss: 0.7173
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7592 - loss: 0.6910
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7682 - loss: 0.6465
Epoch 11/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7623 - loss: 0.6501
Epoch 12/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7711 - lo

In [10]:
testSamples = np.random.randint(1, 201, size=5).reshape(-1, 1)  # Generowanie testowych liczb
testSamples_padded = pad_sequences(testSamples, maxlen=OUTPUT_SEQUENCE_LEN, padding='post')
predictions = model.predict(testSamples_padded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step


In [11]:
for i, prediction in enumerate(predictions):
    decoded = ''
    for j in range(OUTPUT_SEQUENCE_LEN):
        char_index = np.argmax(prediction[j])  # Największa wartość w wektorze to wybrany znak
        decoded += codes[char_index]
    print(f'Liczba: {testSamples[i][0]} -> Przewidziane: {decoded.strip()}')

Liczba: 200 -> Przewidziane: CC
Liczba: 168 -> Przewidziane: CLXVIII
Liczba: 11 -> Przewidziane: XI
Liczba: 124 -> Przewidziane: CXXIV
Liczba: 19 -> Przewidziane: XIX


ZADANIE 2

In [36]:
import random
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping

In [44]:
ROMAN_CHARS = ['I', 'V', 'X', 'L', 'C', 'D', 'M']
ROMAN_CHAR_INDEX = {char: i + 1 for i, char in enumerate(ROMAN_CHARS)}
MAX_LEN = 7

def arabic_to_roman(num):
    roman_numerals = {
        1: "I", 4: "IV", 5: "V", 9: "IX", 10: "X", 40: "XL", 50: "L",
        90: "XC", 100: "C", 400: "CD", 500: "D", 900: "CM", 1000: "M"
    }
    result = ""
    for value, roman in sorted(roman_numerals.items(), key=lambda x: -x[0]):
        while num >= value:
            result += roman
            num -= value
    return result

def generate_roman_dataset(min_value=1, max_value=200, num_samples=1000):
    data = []
    for _ in range(num_samples):
        value = random.randint(min_value, max_value)
        roman = arabic_to_roman(value)
        data.append((roman, value))
    return data


In [45]:
def prepare_data(dataset):
    inputs, outputs = zip(*dataset)
    input_sequences = [[ROMAN_CHAR_INDEX[char] for char in seq] for seq in inputs]
    padded_inputs = pad_sequences(input_sequences, padding='post')

    output_values = np.array(outputs)

    scaler = MinMaxScaler()
    output_values_scaled = scaler.fit_transform(output_values.reshape(-1, 1))

    return padded_inputs, output_values_scaled, scaler

In [47]:
dataset = generate_roman_dataset()
x, y, scaler = prepare_data(dataset)

model = Sequential([
 Embedding(input_dim=len(ROMAN_CHARS) + 1, output_dim=8, input_length=x.shape[1]),
 LSTM(32, return_sequences=False),
 Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [48]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(x, y, epochs=200, batch_size=32, validation_split=0.2,
                    callbacks=[early_stop], verbose=1)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.2519 - mae: 0.4170 - val_loss: 0.1169 - val_mae: 0.2993
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0859 - mae: 0.2473 - val_loss: 0.0810 - val_mae: 0.2511
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0651 - mae: 0.2219 - val_loss: 0.0404 - val_mae: 0.1695
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0204 - mae: 0.1139 - val_loss: 0.0034 - val_mae: 0.0443
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0027 - mae: 0.0436 - val_loss: 0.0016 - val_mae: 0.0305
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0015 - mae: 0.0318 - val_loss: 0.0012 - val_mae: 0.0256
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011 - mae: 0.0251 - val_loss: 9.4868e-04 - val_mae: 0.0249
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.9408e-04 - mae: 0.0220 - val_loss: 9.4168e-04 - val_mae: 0.0204
Epoch 9/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 

In [50]:
def predict_roman_to_arabic(roman_str, scaler):
    input_seq = [[ROMAN_CHAR_INDEX[char] for char in roman_str]]
    padded_input = pad_sequences(input_seq, padding='post', maxlen=x.shape[1])
    prediction_scaled = model.predict(padded_input, verbose=0)

    prediction = scaler.inverse_transform(prediction_scaled)
    return round(prediction[0][0])

def evaluate_model(test_data, scaler):
    results = []
    for roman, actual in test_data:
        predicted = predict_roman_to_arabic(roman, scaler)
        results.append({
            'Roman': roman,
            'Actual Arabic': actual,
            'Predicted Arabic': predicted,
            'Error': abs(predicted - actual)
        })
    return pd.DataFrame(results)

test_dataset = generate_roman_dataset(num_samples=200)
evaluation_results = evaluate_model(test_dataset, scaler)

print("\nWyniki:")
print(evaluation_results.head(20))


Wyniki:
      Roman  Actual Arabic  Predicted Arabic  Error
0    CLXXII            172               172      0
1   LXXXVII             87                87      0
2       CLX            160               160      0
3       LIX             59                60      1
4        II              2                 2      0
5      CXXV            125               125      0
6    LXXIII             73                73      0
7      XLIV             44                46      2
8    CLXXIV            174               175      1
9      CXIX            119               120      1
10    CLVII            157               157      0
11     CLIV            154               156      2
12   CXCVII            197               197      0
13  CLXVIII            168               169      1
14       XX             20                20      0
15     XXIV             24                25      1
16      CIX            109               110      1
17       XL             40                41      1
18 

PODSUMOWANIE:
- zadanie 1: model udało się wytrenować na danych, osiągając doskonałe wyniki z minimalnym błędem.
- zadanie 2: konwersja liczb rzymskich na arabskie dawało bardzo dokładne przewidywania, gdzie model potrafił rozpoznać liczby rzymskie i dokładnie konwertować je na wartości arabskie. Błąd dla większości danych był bardzo niski (zaledwie kilka przypadków różnicy o 1 czy 2). Błąd dla większości testów wynosił 0 lub 1, co jest doskonałym wynikiem dla tego typu problemu.

Zastosowanie LSTM w tym przypadku sprawdza się w doskonały sposób do przetwarzania sekwencji danych wejściowych. Dzięki poprawnemu preprocessingu (wektoryzacja liczb rzymskich, padding) oraz normalizacji wyników, model uczy się efektywnie, a predykcje są dokładne. To rozwiązanie stanowi przykład użycia RNN, LSTM i normalizacji danych w praktycznym problemie przetwarzania sekwencji. Dzięki tej metodzie, potrafimy skutecznie przewidywać konkretne liczby arabskie na podstawie znanych reguł liczb rzymskich.